# calculating seasonal composites of atmospheric responses to CP and EP La Nina

In [155]:
import warnings
warnings.filterwarnings('ignore')

In [156]:
import numpy as np
import os
import xarray as xr
import xcdat as xc
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm as BM
import pandas as pd
import matplotlib as mpl
import matplotlib.ticker as mticker
import netCDF4
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [157]:
from scipy import stats

In [158]:
from functions import preproc_funcs as funcs

In [159]:
mpl.rcParams['font.family'] = 'Avenir'
mpl.rcParams['font.size'] = 12
# Edit axes parameters
mpl.rcParams['axes.linewidth'] = 2.0
# Tick properties
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['xtick.minor.size'] = 3
mpl.rcParams['xtick.major.width'] = 1
mpl.rcParams['xtick.direction'] = 'out'
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['ytick.minor.size'] = 3
mpl.rcParams['ytick.major.width'] = 1
mpl.rcParams['ytick.direction'] = 'out'

In [160]:
pres = xr.open_mfdataset('./data/20CR/slp/prmsl.mon.mean.nc', parallel=True)
tas = xr.open_mfdataset('./data/20CR/tas/air.2m.mon.mean.nc', parallel=True)
pr = xr.open_mfdataset('./data/20CR/precip/prate.mon.mean.nc', parallel=True)
uwnd = xr.open_mfdataset('./data/20CR/wind/uwnd.hgtAbvSfc.mon.mean.nc', parallel=True)
vwnd = xr.open_mfdataset('./data/20CR/wind/vwnd.hgtAbvSfc.mon.mean.nc', parallel=True)
omega = xr.open_mfdataset('./data/20CR/wind/omega.mon.mean.nc', parallel=True)

HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 0:
  #000: H5A.c line 528 in H5Aopen_by_name(): can't open attribute
    major: Attribute
    minor: Can't open object
  #001: H5VLcallback.c line 1091 in H5VL_attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #002: H5VLcallback.c line 1058 in H5VL__attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #003: H5VLnative_attr.c line 130 in H5VL__native_attr_open(): can't open attribute
    major: Attribute
    minor: Can't open object
  #004: H5Aint.c line 545 in H5A__open_by_name(): unable to load attribute info from object header
    major: Attribute
    minor: Unable to initialize object
  #005: H5Oattribute.c line 476 in H5O__attr_open_by_name(): can't open attribute
    major: Attribute
    minor: Can't open object
  #006: H5Adense.c line 394 in H5A__dense_open(): can't locate attribute in name index
    major: Attribute
    minor: Object not 

In [161]:
gz500 = xr.open_mfdataset('./data/20CR/gph/hgt.mon.mean_500.nc', parallel=True)

HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 0:
  #000: H5A.c line 528 in H5Aopen_by_name(): can't open attribute
    major: Attribute
    minor: Can't open object
  #001: H5VLcallback.c line 1091 in H5VL_attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #002: H5VLcallback.c line 1058 in H5VL__attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #003: H5VLnative_attr.c line 130 in H5VL__native_attr_open(): can't open attribute
    major: Attribute
    minor: Can't open object
  #004: H5Aint.c line 545 in H5A__open_by_name(): unable to load attribute info from object header
    major: Attribute
    minor: Unable to initialize object
  #005: H5Oattribute.c line 494 in H5O__attr_open_by_name(): can't locate attribute: '_QuantizeBitGroomNumberOfSignificantDigits'
    major: Attribute
    minor: Object not found
HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 0:
  #000: H5A.c line 528 in H5Ao

In [162]:
sst = xr.open_mfdataset('./data/HadISST_sst.nc', parallel=True)

In [163]:
uwnd_levels = xr.open_mfdataset('./data/20CR/wind/uwnd.mon.mean.nc', parallel=True)

HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 0:
  #000: H5A.c line 528 in H5Aopen_by_name(): can't open attribute
    major: Attribute
    minor: Can't open object
  #001: H5VLcallback.c line 1091 in H5VL_attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #002: H5VLcallback.c line 1058 in H5VL__attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #003: H5VLnative_attr.c line 130 in H5VL__native_attr_open(): can't open attribute
    major: Attribute
    minor: Can't open object
  #004: H5Aint.c line 545 in H5A__open_by_name(): unable to load attribute info from object header
    major: Attribute
    minor: Unable to initialize object
  #005: H5Oattribute.c line 476 in H5O__attr_open_by_name(): can't open attribute
    major: Attribute
    minor: Can't open object
  #006: H5Adense.c line 394 in H5A__dense_open(): can't locate attribute in name index
    major: Attribute
    minor: Object not 

In [164]:
mixed_years = np.array([1908,1910,1916,1917,1922,1949,1995])
cp_years = np.array([1903,1909,1924,1933,1938,1942,1950,1954,1955,1964,1970,1971,1973,1974,1975,1984,1988,1998,1999,2000,2007,2010,2011,2020,2022])
ep_years = np.array([1906,1962,1967,2017,2021])
neut_years = np.array([1901,1907,1912,1921,1923,1926,1927,1928,1929,1931,1932,1934,1935,1936,1943,1944,1945,1946,1947,1948,1952,1953,1958,1959,1960,1978,1980,1981,1983,1989,1990,1993,1996,2001,2008,2012,2013,2016,2019])

In [165]:
tas_anom = funcs.calc_anom((tas.air), (tas.air))

In [166]:
sst_anom = funcs.calc_anom((sst.sst), (sst.sst))

In [167]:
pr_anom = funcs.calc_anom((pr.prate*86400), ((pr.prate*86400)))

In [168]:
import regionmask

In [15]:
land_mask = regionmask.defined_regions.natural_earth_v5_0_0.land_110.mask(pr_anom)
# pr_anom_land = pr_anom.isel(mask_ocean = 1)
land_mask

<xarray.DataArray 'mask' (lat: 181, lon: 360)>
array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * lat      (lat) float32 -90.0 -89.0 -88.0 -87.0 -86.0 ... 87.0 88.0 89.0 90.0
  * lon      (lon) float32 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0

In [16]:
pr_anom_land = pr_anom.where(~np.isnan(land_mask))
tas_anom_ocean = tas_anom.where(np.isnan(land_mask))

In [169]:
def get_composite(val, season=[12,1,2], years=np.arange(1960, 1991, 1), standardise=True):
    val_seasonal_mean_det = funcs.detrend_rolling_window(val.where(val.time.dt.month.isin(season), drop=True).resample(time = 'AS-JUN').mean('time').chunk(dict(time=-1, lat=5, lon=10)), window_size=15)
    if standardise:
        return val_seasonal_mean_det.where(val_seasonal_mean_det.time.dt.year.isin(years), drop=True).mean('time')/val_seasonal_mean_det.std()
    else:
        return val_seasonal_mean_det.where(val_seasonal_mean_det.time.dt.year.isin(years), drop=True).mean('time')



def get_composite_omega(val, season=[12,1,2], years=np.arange(1960, 1991, 1), standardise=True):
    val_seasonal_mean_det = funcs.detrend_rolling_window(val.where(val.time.dt.month.isin(season), drop=True).resample(time = 'AS-JUN').mean('time').chunk(dict(time=-1, lon=10)), window_size=15)
    if standardise:
        return val_seasonal_mean_det.where(val_seasonal_mean_det.time.dt.year.isin(years), drop=True).mean('time')/val_seasonal_mean_det.std()
    else:
        return val_seasonal_mean_det.where(val_seasonal_mean_det.time.dt.year.isin(years), drop=True).mean('time')


def get_individual_years(val, season=[12,1,2], years=np.arange(1960, 1991, 1), standardise=True):
    val_seasonal_mean_det = funcs.detrend_rolling_window(val.where(val.time.dt.month.isin(season), drop=True).resample(time = 'AS-JUN').mean('time').chunk(dict(time=-1)), window_size=15)
    if standardise:
        return val_seasonal_mean_det.where(val_seasonal_mean_det.time.dt.year.isin(years), drop=True)/val_seasonal_mean_det.std()
    else:
        return val_seasonal_mean_det.where(val_seasonal_mean_det.time.dt.year.isin(years), drop=True)


In [34]:
pr_ep = get_individual_years(pr_anom, season=[12,1,2], years=ep_years, standardise=True).load()

In [35]:
pr_cp = get_individual_years(pr_anom, season=[12,1,2], years=cp_years, standardise=True).load()

In [36]:
pr_mixed = get_individual_years(pr_anom, season=[12,1,2], years=mixed_years, standardise=True).load()

In [37]:
pr_neut = get_individual_years(pr_anom, season=[12,1,2], years=neut_years, standardise=True).load()

In [38]:
pr_ep.to_netcdf('./data/res/indiv_years/djf/pr_ep.nc')
pr_cp.to_netcdf('./data/res/indiv_years/djf/pr_cp.nc')
pr_mixed.to_netcdf('./data/res/indiv_years/djf/pr_mixed.nc')
pr_neut.to_netcdf('./data/res/indiv_years/djf/pr_neut.nc')

In [39]:
uwnd_anom = funcs.calc_anom(uwnd.sel(level=100), uwnd.sel(level=100))
vwnd_anom = funcs.calc_anom(vwnd.sel(level=100), vwnd.sel(level=100))

In [40]:
# with ProgressBar():
#     uwnd_plot_cp = get_composite(uwnd_anom, season=[9,10,11,12,1,2,3], years=cp_years, standardise=False).load()
#     vwnd_plot_cp = get_composite(vwnd_anom, season=[9,10,11,12,1,2,3], years=cp_years, standardise=False).load()

In [56]:
uwnd_ep = get_individual_years(uwnd_anom, season=[12,1,2], years=ep_years, standardise=False).load()

In [57]:
uwnd_cp = get_individual_years(uwnd_anom, season=[12,1,2], years=cp_years, standardise=False).load()

In [58]:
uwnd_mixed = get_individual_years(uwnd_anom, season=[12,1,2], years=mixed_years, standardise=False).load()

In [59]:
uwnd_neut = get_individual_years(uwnd_anom, season=[12,1,2], years=neut_years, standardise=False).load()

In [60]:
uwnd_ep.to_netcdf('./data/res/indiv_years/djf/uwnd_ep.nc')
uwnd_cp.to_netcdf('./data/res/indiv_years/djf/uwnd_cp.nc')
uwnd_mixed.to_netcdf('./data/res/indiv_years/djf/uwnd_mixed.nc')
uwnd_neut.to_netcdf('./data/res/indiv_years/djf/unwnd_neut.nc')

In [79]:
vwnd_ep = get_individual_years(vwnd_anom, season=[12,1,2], years=ep_years, standardise=False).load()

In [80]:
vwnd_cp = get_individual_years(vwnd_anom, season=[12,1,2], years=cp_years, standardise=False).load()

In [81]:
vwnd_mixed = get_individual_years(vwnd_anom, season=[12,1,2], years=mixed_years, standardise=False).load()

In [82]:
vwnd_neut = get_individual_years(vwnd_anom, season=[12,1,2], years=neut_years, standardise=False).load()

In [83]:
vwnd_ep.to_netcdf('./data/res/indiv_years/djf/vwnd_ep.nc')
vwnd_cp.to_netcdf('./data/res/indiv_years/djf/vwnd_cp.nc')
vwnd_mixed.to_netcdf('./data/res/indiv_years/djf/vwnd_mixed.nc')
vwnd_neut.to_netcdf('./data/res/indiv_years/djf/vwnd_neut.nc')

In [84]:
pres_anom = funcs.calc_anom(pres.prmsl, pres.prmsl)

In [173]:
omega_anom = funcs.calc_anom(omega.sel(lat = slice(-10, 10)).mean('lat'), omega.sel(lat = slice(-10,10)).mean('lat'))

In [170]:
uwnd_levels = xr.open_mfdataset('./data/20CR/wind/uwnd.mon.mean.nc', parallel=True)
uwnd_levels

HDF5-DIAG: Error detected in HDF5 (1.12.2) thread 0:
  #000: H5A.c line 528 in H5Aopen_by_name(): can't open attribute
    major: Attribute
    minor: Can't open object
  #001: H5VLcallback.c line 1091 in H5VL_attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #002: H5VLcallback.c line 1058 in H5VL__attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #003: H5VLnative_attr.c line 130 in H5VL__native_attr_open(): can't open attribute
    major: Attribute
    minor: Can't open object
  #004: H5Aint.c line 545 in H5A__open_by_name(): unable to load attribute info from object header
    major: Attribute
    minor: Unable to initialize object
  #005: H5Oattribute.c line 476 in H5O__attr_open_by_name(): can't open attribute
    major: Attribute
    minor: Can't open object
  #006: H5Adense.c line 394 in H5A__dense_open(): can't locate attribute in name index
    major: Attribute
    minor: Object not 

<xarray.Dataset>
Dimensions:    (level: 28, lat: 181, lon: 360, time: 2520, nbnds: 2)
Coordinates:
  * level      (level) float32 1e+03 975.0 950.0 925.0 ... 20.0 10.0 5.0 1.0
  * lat        (lat) float32 -90.0 -89.0 -88.0 -87.0 ... 87.0 88.0 89.0 90.0
  * lon        (lon) float32 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
  * time       (time) datetime64[ns] 1806-01-01 1806-02-01 ... 2015-12-01
Dimensions without coordinates: nbnds
Data variables:
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(2520, 2), meta=np.ndarray>
    uwnd       (time, level, lat, lon) float32 dask.array<chunksize=(2520, 28, 181, 360), meta=np.ndarray>
Attributes: (12/24)
    Conventions:               CF-1.2
    title:                     Monthly NOAA/CIRES/DOE 20th Century Reanalysis V3
    comments:                  Data are from \nNOAA/CIRES/DOE 20th Century Re...
    platform:                  Model
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    license:                   These data are available free of charge under ...
    ...                        ...
    citation1:                 Slivinski, L. C, G. P. Compo, J. S. Whitaker, ...
    References:                https://www.psl.noaa.gov/data/gridded/data.20t...
    creator_name:              NOAA/PSL
    institution:               NOAA Physical Sciences Laboratory & CU/CIRES \...
    contact:                   psl.data@noaa.gov
    citation:                  Compo,G.P. <https://www.psl.noaa.gov/people/gi...

In [172]:
uwnd_levels_anom = funcs.calc_anom(uwnd_levels.sel(lat = slice(-10, 10)).mean('lat'), uwnd_levels.sel(lat = slice(-10,10)).mean('lat'))

In [86]:
gz500_anom = funcs.calc_anom(gz500.hgt, gz500.hgt)

In [104]:
pres_ep = get_individual_years(pres_anom, season=[12,1,2], years=ep_years, standardise=False).load()

In [105]:
pres_cp = get_individual_years(pres_anom, season=[12,1,2], years=cp_years, standardise=False).load()

In [106]:
pres_mixed = get_individual_years(pres_anom, season=[12,1,2], years=mixed_years, standardise=False).load()

In [108]:
pres_neut = get_individual_years(pres_anom, season=[12,1,2], years=neut_years, standardise=False).load()

In [109]:
pres_ep.to_netcdf('./data/res/indiv_years/djf/pres_ep.nc')
pres_cp.to_netcdf('./data/res/indiv_years/djf/pres_cp.nc')
pres_mixed.to_netcdf('./data/res/indiv_years/djf/pres_mixed.nc')
pres_neut.to_netcdf('./data/res/indiv_years/djf/pres_neut.nc')

In [126]:
omega_ep = get_individual_years(omega_anom, season=[12,1,2], years=ep_years, standardise=False).load()

In [127]:
omega_cp = get_individual_years(omega_anom, season=[12,1,2], years=cp_years, standardise=False).load()

In [128]:
omega_mixed = get_individual_years(omega_anom, season=[12,1,2], years=mixed_years, standardise=False).load()

In [129]:
omega_neut = get_individual_years(omega_anom, season=[12,1,2], years=neut_years, standardise=False).load()

In [130]:
omega_ep.to_netcdf('./data/res/indiv_years/djf/omega_ep.nc')
omega_cp.to_netcdf('./data/res/indiv_years/djf/omega_cp.nc')
omega_mixed.to_netcdf('./data/res/indiv_years/djf/omega_mixed.nc')
omega_neut.to_netcdf('./data/res/indiv_years/djf/omega_neut.nc')

In [174]:
uwnd_levels_ep = get_individual_years(uwnd_levels_anom, season=[3,4,5], years=ep_years, standardise=False).load()

In [175]:
uwnd_levels_cp = get_individual_years(uwnd_levels_anom, season=[3,4,5], years=cp_years, standardise=False).load()

In [176]:
uwnd_levels_mixed = get_individual_years(uwnd_levels_anom, season=[3,4,5], years=mixed_years, standardise=False).load()

In [177]:
uwnd_levels_neut = get_individual_years(uwnd_levels_anom, season=[3,4,5], years=neut_years, standardise=False).load()

In [178]:
uwnd_levels_ep.to_netcdf('./data/res/indiv_years/mam/uwnd_levels_ep.nc')
uwnd_levels_cp.to_netcdf('./data/res/indiv_years/mam/uwnd_levels_cp.nc')
uwnd_levels_mixed.to_netcdf('./data/res/indiv_years/mam/uwnd_levels_mixed.nc')
uwnd_levels_neut.to_netcdf('./data/res/indiv_years/mam/uwnd_levels_neut.nc')

In [150]:
gz500_ep = get_individual_years(gz500_anom, season=[12,1,2], years=ep_years, standardise=False).load()

In [151]:
gz500_cp = get_individual_years(gz500_anom, season=[12,1,2], years=cp_years, standardise=False).load()

In [152]:
gz500_mixed = get_individual_years(gz500_anom, season=[12,1,2], years=mixed_years, standardise=False).load()

In [153]:
gz500_neut = get_individual_years(gz500_anom, season=[12,1,2], years=neut_years, standardise=False).load()

In [154]:
gz500_ep.to_netcdf('./data/res/indiv_years/djf/gz500_ep.nc')
gz500_cp.to_netcdf('./data/res/indiv_years/djf/gz500_cp.nc')
gz500_mixed.to_netcdf('./data/res/indiv_years/djf/gz500_mixed.nc')
gz500_neut.to_netcdf('./data/res/indiv_years/djf/gz500_neut.nc')